In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory

from dotenv import load_dotenv

In [28]:
load_dotenv()

True

In [29]:
# before this, run the following: docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
REDIS_URL = "redis://localhost:6379/0"

In [30]:
chat = ChatGroq(
    temperature=0,
    model='llama-3.3-70b-versatile'
)

system = """
You are an investing professional, specializing in stock-picking. Answer the user's questions on investing in detail.
Given a chat history and the latest user question which might reference context in the chat history, answer the question
to the best of your ability. If you do not know the answer, just return "I don't know".
"""

def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]
)

chain = prompt | chat

with_message_history = RunnableWithMessageHistory(
    chain,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [32]:
with_message_history.invoke(
    {"ability": "investing", "input": "Did I ask you about cosine before?"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content="No, you didn't ask me about cosine before. This conversation just started, and your first question was about whether you asked me about cosine before.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 170, 'total_tokens': 201, 'completion_time': 0.112727273, 'prompt_time': 0.010632587, 'queue_time': 0.233271133, 'total_time': 0.12335986}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_76dc6cf67d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd6c7f43-c4ee-426b-b20b-dc0eaada4ede-0', usage_metadata={'input_tokens': 170, 'output_tokens': 31, 'total_tokens': 201})

In [33]:
history = get_message_history('foobar')
history.clear()

In [35]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_message_history at 0x00000230E1036520>, input_messages_key='input', history_messages_key='history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [ ]:
# convert into a class

class InvestingChatBot:

    # define default system prompt
    system_prompt = """
                    You are an investing professional, specializing in stock-picking. Answer the user's questions on investing in detail.
                    Given a chat history and the latest user question which might reference context in the chat history, answer the question
                    to the best of your ability. If you do not know the answer, just return "I don't know".
                    """

    def __init__(self, model: str, temperature: float, session_id: str):

        '''Initialize the model, temperature, and the session id'''

        self.model = model
        self.temperature = temperature
        self.session_id = session_id

        self.chat = self.create_llm_chain_with_history()
        

    def get_message_history(self) -> RedisChatMessageHistory:
        '''
        Get the message history of the current session
        '''
        return RedisChatMessageHistory(self.session_id, url=REDIS_URL)
    
    def clear_history(self):
        '''
        Clears the message history for the current session
        '''
        history = self.get_message_history()
        history.clear()
    
    def create_llm_chain_with_history(self) -> RunnableWithMessageHistory:
        '''
        Create a runnable with message history. Message history saved to redis database for persistence
        '''

        chat = ChatGroq(
            temperature=self.temperature,
            model=self.model
        )

        # define the prompt
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_prompt),
                MessagesPlaceholder(variable_name="history"), # add in chat history
                ("human", "{input}")
            ]
        )
        
        # create the chain
        chain = prompt | chat

        # add message history to the llm
        chain_with_message_history = RunnableWithMessageHistory(
            chain,
            self.get_message_history, # add in method to get message history
            input_messages_key="input",
            history_messages_key="history",
        )

        return chain_with_message_history
    
    def prompt(self, ability: str, input: str) -> str:
        '''
        Prompt the LLM, get a response in string
        '''
        
        response = self.chat.invoke(
            {"ability": ability, "input": input},
            config={"configurable": {"session_id": self.session_id}},
        )

        return response.content

In [ ]:
investingChatBot = InvestingChatBot(
    "llama-3.3-70b-versatile",
    0,
    'jiahao'
)

r2 = investingChatBot.prompt(
    ability='investing',
    input='What is my name?'
)

print(r2)

# investingChatBot.clear_history()

r3 = investingChatBot.prompt(
    ability='investing',
    input='What is my name?'
)

print(r3)

I don't know.
I don't know. This conversation just started, and I haven't received any information about your name. If you'd like to share it with me, I'd be happy to know, but for now, I don't have that information. By the way, are you interested in discussing investing or stock-picking strategies?
